Author : `Muhammad Abdullah`
Email : `mrabdullah0303@gmail.com`
---
About This Notebook : In this Notebook i will Code a Program to Select best Model and Its Parameters by HyperparameteringTunning . 
I used `Tips` and `Diamonds` Dataset in This Notebook . 

U can Ask Questions Related to this Notebook . 

- Importing Libraries 

In [1]:
# Import Libraires 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Train test Split 
from sklearn.model_selection import train_test_split
# Models 
from sklearn.naive_bayes import GaussianNB , BernoulliNB , MultinomialNB
from sklearn.tree import DecisionTreeClassifier , DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier ,RandomForestRegressor , AdaBoostRegressor
from xgboost import XGBClassifier , XGBRegressor
from sklearn.linear_model import LinearRegression ,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor , KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor , GradientBoostingClassifier
from sklearn.svm import SVC , SVR
#metrics
from sklearn.metrics import mean_squared_error , mean_absolute_error , r2_score , classification_report , accuracy_score , f1_score , precision_score
#import grid search cv for cross validation
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV
# import preprocessors
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.preprocessing import QuantileTransformer , PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# Remove Warning 
import warnings
warnings.filterwarnings('ignore')
# Saving Model 
import pickle

# 1. Regressor 

In [2]:
# Load Dataset 
df = sns.load_dataset('tips')
# select features and variables
X = df.drop('tip', axis=1)
y = df['tip']

# label encode Input  variables
le = LabelEncoder()
X['sex'] = le.fit_transform(X['sex'])
X['smoker'] = le.fit_transform(X['smoker'])
X['day'] = le.fit_transform(X['day'])
X['time'] = le.fit_transform(X['time'])

# # Quantile Transform , PowerTransformer
# qt = QuantileTransformer(output_distribution='normal', random_state=42, n_quantiles=500)
# X['total_bill'] = qt.fit_transform(X[['total_bill']])

# #Standard Scaler
# scaler = StandardScaler()
# X['total_bill'] = scaler.fit_transform(X[['total_bill']])

# split the data into train and test data with 80% training dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 1.1 RandomiedSearchCv 

In [5]:

# # Create a dictionaries of list of models to evaluate performance with hyperparameters
models = { 
          'LinearRegression' : (LinearRegression(), {}),
          'SVR' : (SVR(), {'kernel': ['rbf', 'poly', 'sigmoid']}),
          'DecisionTreeRegressor' : (DecisionTreeRegressor(random_state=42), {'max_depth': [None, 5, 10]}),
          'RandomForestRegressor' : (RandomForestRegressor(random_state=42), {'n_estimators': [10, 100]}),
          'KNeighborsRegressor' : (KNeighborsRegressor(), {'n_neighbors': np.arange(3, 100, 2)}),
          'GradientBoostingRegressor' : (GradientBoostingRegressor(random_state=42),{'n_estimators': [10, 100]}),
          'XGBRegressor' : (XGBRegressor(), {'n_estimators': [10, 100]}),  
          'AdaBoostRegressor': (AdaBoostRegressor(random_state=42), {'n_estimators': [10, 100]}),        
          }
model_scores = []
# For loop to iterate over the models
for name, (model, params) in models.items():
    # create a pipline
    pipeline = RandomizedSearchCV(model, params, cv=5 , verbose=1 , n_jobs=-1)
    
    # fit the pipeline
    pipeline.fit(X_train, y_train)
    
    # make prediction from each model
    y_pred = pipeline.predict(X_test)
    # Metric
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    best_parameter = pipeline.best_params_
    model_scores.append((name,r2 , mae , mse,best_parameter))
# selecting the best model from all above models with evaluation metrics sorting method
sorted_models = sorted(model_scores, key=lambda x: x[1], reverse=False)
# # Printing Each model with evaluation metrics
for model in sorted_models:
    print('R_2 for', f"{model[0]} is {model[1]: .2f}")
print('\n')
for model in sorted_models:
    print('MAE for', f"{model[0]} is {model[2]: .2f}")
print('\n')
for model in sorted_models:
    print('MSE for', f"{model[0]} is {model[3]: .2f}")


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
R_2 for SVR is -0.17
R_2 for RandomForestRegressor is  0.23
R_2 for AdaBoostRegressor is  0.23
R_2 for DecisionTreeRegressor is  0.30
R_2 for GradientBoostingRegressor is  0.35
R_2 for LinearRegression is  0.44
R_2 for KNeighborsRegressor is  0.47
R_2 for XGBRegressor is  0.47


MAE for SVR is  0.89
MAE for RandomForestRegressor is  0.78
MAE for AdaBoostRegressor is  0.78
MAE for DecisionTreeRegressor is  0.72
MAE for GradientBoostingRegressor is  0.77
MAE for LinearRegression is  0.67
MAE for KNeighborsRegressor i

- Best Parameter and Best Model 

In [19]:

# Best Parameter for each model
for model in sorted_models:
    print('Best Parameters for', f"{model[0]} is {model[4]}")
print('\n')
# Selecting the best model based on R2
best_r2_model = max(model_scores, key=lambda x: x[1])
print('Best model based on R2 is', f"{best_r2_model[0]} with R2 of {best_r2_model[1]:.2f}")
print(f'Best Parameters: {best_r2_model[4]}')
# Selecting the best model based on MAE
best_mae_model = min(model_scores, key=lambda x: x[2])
print('Best model based on MAE is', f"{best_mae_model[0]} with MAE of {best_mae_model[2]:.2f}")
print(f'Best Parameters: {best_mae_model[4]}')
# Selecting the best model based on MSE 
best_mse_model = min(model_scores, key=lambda x: x[3])
print('Best model based on MSE is', f"{best_mse_model[0]} with MSE of {best_mse_model[3]:.2f}")
print(f'Best Parameters: {best_mse_model[4]}')

Best Parameters for SVR is {'kernel': 'poly'}
Best Parameters for RandomForestRegressor is {'n_estimators': 100}
Best Parameters for AdaBoostRegressor is {'n_estimators': 10}
Best Parameters for DecisionTreeRegressor is {'max_depth': 5}
Best Parameters for GradientBoostingRegressor is {'n_estimators': 10}
Best Parameters for LinearRegression is {}
Best Parameters for XGBRegressor is {'n_estimators': 10}
Best Parameters for KNeighborsRegressor is {'n_neighbors': 55}


Best model based on R2 is KNeighborsRegressor with R2 of 0.49
Best Parameters: {'n_neighbors': 55}
Best model based on MAE is XGBRegressor with MAE of 0.65
Best Parameters: {'n_estimators': 10}
Best model based on MSE is KNeighborsRegressor with MSE of 0.63
Best Parameters: {'n_neighbors': 55}


## 1.2 Saving Model 

In [20]:
# Selecting the best model based on R2
best_r2_model = max(model_scores, key=lambda x: x[1])
best_model_name = best_r2_model[0]
best_model_instance = models[best_model_name][0]
print(best_model_name)
# export 
pickle.dump(best_model_instance,open('pipe.pkl','wb'))
# Load Model 
svr = pickle.load(open('pipe.pkl','rb'))
# Fit Model 
svr.fit(X_train, y_train)
# Assume user input
test_input2 = np.array([15, 1, 0, 3, 1, 1],dtype=object).reshape(1,6)
# Predicting the output
svr.predict(test_input2)

KNeighborsRegressor


array([2.11])

In [21]:
# Selecting the best model based on MAE
best_mae_model = min(model_scores, key=lambda x: x[2])
best_model_name = best_mae_model[0]
best_model_instance = models[best_model_name][0]
print(best_model_name)
# export 
pickle.dump(best_model_instance,open('pipe.pkl','wb'))
# Load Model 
svr = pickle.load(open('pipe.pkl','rb'))
# Fit Model 
svr.fit(X_train, y_train)
# Assume user input
test_input2 = np.array([15, 1, 0, 3, 1, 1],dtype=object).reshape(1,6)
# Predicting the output
svr.predict(test_input2)

XGBRegressor


array([2.5676944], dtype=float32)

In [22]:
# Selecting the best model based on MSE
best_mse_model = min(model_scores, key=lambda x: x[3])
best_model_name = best_mse_model[0]
best_model_instance = models[best_model_name][0]
print(best_model_name)
# export 
pickle.dump(best_model_instance,open('pipe.pkl','wb'))
# Load Model 
svr = pickle.load(open('pipe.pkl','rb'))
# Fit Model 
svr.fit(X_train, y_train)
# Assume user input
test_input2 = np.array([15, 1, 0, 3, 1, 1],dtype=object).reshape(1,6)
# Predicting the output
svr.predict(test_input2)

KNeighborsRegressor


array([2.11])

- Using RandomizedCvSearch The Best Performing Model On the base of R2 and MAE  is `SVR` and One the Base of MSE `KNeighborsRegressor` is Performing Well  On `Diamonds` Dataset 
- `Best Parameters for SVR is {'kernel': 'poly', 'gamma': 0.01, 'epsilon': 0.001, 'C': 1}`

## 1.2 GridCvSearch 

In [23]:

# # Create a dictionaries of list of models to evaluate performance with hyperparameters
models = { 
          'LinearRegression' : (LinearRegression(), {}),
          'SVR' : (SVR(), {'kernel': ['rbf', 'poly', 'sigmoid']}),
          'DecisionTreeRegressor' : (DecisionTreeRegressor(random_state=42), {'max_depth': [None, 5, 10]}),
          'RandomForestRegressor' : (RandomForestRegressor(random_state=42), {'n_estimators': [10, 100]}),
          'KNeighborsRegressor' : (KNeighborsRegressor(), {'n_neighbors': np.arange(3, 100, 2)}),
          'GradientBoostingRegressor' : (GradientBoostingRegressor(random_state=42),{'n_estimators': [10, 100]}),
          'XGBRegressor' : (XGBRegressor(), {'n_estimators': [10, 100]}),  
          'AdaBoostRegressor': (AdaBoostRegressor(random_state=42), {'n_estimators': [10, 100]}),        
          }

model_scores = []
# For loop to iterate over the models
for name, (model, params) in models.items():
    # create a pipline
    pipeline = GridSearchCV(model, params, cv=5 , n_jobs=-1)
    
    # fit the pipeline
    pipeline.fit(X_train, y_train)
    
    # make prediction from each model
    y_pred = pipeline.predict(X_test)
    # Metric
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    best_parameter = pipeline.best_params_
    model_scores.append((name,r2 , mae , mse,best_parameter))

# selecting the best model from all above models with evaluation metrics sorting method
sorted_models = sorted(model_scores, key=lambda x: x[1], reverse=False)

# Printing Each model with evaluation metrics
for model in sorted_models:
    print('R_2 for', f"{model[0]} is {model[1]: .2f}")
print('\n')
for model in sorted_models:
    print('MAE for', f"{model[0]} is {model[2]: .2f}")
print('\n')
for model in sorted_models:
    print('MSE for', f"{model[0]} is {model[3]: .2f}")


R_2 for SVR is -0.17
R_2 for RandomForestRegressor is  0.23
R_2 for AdaBoostRegressor is  0.23
R_2 for DecisionTreeRegressor is  0.30
R_2 for GradientBoostingRegressor is  0.35
R_2 for LinearRegression is  0.44
R_2 for KNeighborsRegressor is  0.47
R_2 for XGBRegressor is  0.47


MAE for SVR is  0.89
MAE for RandomForestRegressor is  0.78
MAE for AdaBoostRegressor is  0.78
MAE for DecisionTreeRegressor is  0.72
MAE for GradientBoostingRegressor is  0.77
MAE for LinearRegression is  0.67
MAE for KNeighborsRegressor is  0.62
MAE for XGBRegressor is  0.65


MSE for SVR is  1.46
MSE for RandomForestRegressor is  0.96
MSE for AdaBoostRegressor is  0.96
MSE for DecisionTreeRegressor is  0.88
MSE for GradientBoostingRegressor is  0.81
MSE for LinearRegression is  0.69
MSE for KNeighborsRegressor is  0.66
MSE for XGBRegressor is  0.66


- BestParameter and Best Model 

In [24]:

# Best Parameter for each model
for model in sorted_models:
    print('Best Parameters for', f"{model[0]} is {model[4]}")
print('\n')
# Selecting the best model based on R2
best_r2_model = max(model_scores, key=lambda x: x[1])
print('Best model based on R2 is', f"{best_r2_model[0]} with R2 of {best_r2_model[1]:.2f}")
print(f'Best Parameters: {best_r2_model[4]}')
# Selecting the best model based on MAE
best_mae_model = min(model_scores, key=lambda x: x[2])
print('Best model based on MAE is', f"{best_mae_model[0]} with MAE of {best_mae_model[2]:.2f}")
print(f'Best Parameters: {best_mae_model[4]}')
# Selecting the best model based on MSE 
best_mse_model = min(model_scores, key=lambda x: x[3])
print('Best model based on MSE is', f"{best_mse_model[0]} with MSE of {best_mse_model[3]:.2f}")
print(f'Best Parameters: {best_mse_model[4]}')

Best Parameters for SVR is {'kernel': 'poly'}
Best Parameters for RandomForestRegressor is {'n_estimators': 100}
Best Parameters for AdaBoostRegressor is {'n_estimators': 10}
Best Parameters for DecisionTreeRegressor is {'max_depth': 5}
Best Parameters for GradientBoostingRegressor is {'n_estimators': 10}
Best Parameters for LinearRegression is {}
Best Parameters for KNeighborsRegressor is {'n_neighbors': 17}
Best Parameters for XGBRegressor is {'n_estimators': 10}


Best model based on R2 is XGBRegressor with R2 of 0.47
Best Parameters: {'n_estimators': 10}
Best model based on MAE is KNeighborsRegressor with MAE of 0.62
Best Parameters: {'n_neighbors': 17}
Best model based on MSE is XGBRegressor with MSE of 0.66
Best Parameters: {'n_estimators': 10}


## 1.2 Saving Best Model and Predicting 

In [25]:
# Selecting the best model based on R2
best_r2_model = max(model_scores, key=lambda x: x[1])
best_model_name = best_r2_model[0]
best_model_instance = models[best_model_name][0]
print(best_model_name)
# export 
pickle.dump(best_model_instance,open('pipe.pkl','wb'))
# Load Model 
svr = pickle.load(open('pipe.pkl','rb'))
# Fit Model 
svr.fit(X_train, y_train)
# Assume user input
test_input2 = np.array([15, 1, 0, 3, 1, 1],dtype=object).reshape(1,6)
# Predicting the output
svr.predict(test_input2)

XGBRegressor


array([2.5676944], dtype=float32)

In [26]:
# Selecting the best model based on MAE
best_mae_model = min(model_scores, key=lambda x: x[2])
best_model_name = best_mae_model[0]
best_model_instance = models[best_model_name][0]
print(best_model_name)
# export 
pickle.dump(best_model_instance,open('pipe.pkl','wb'))
# Load Model 
svr = pickle.load(open('pipe.pkl','rb'))
# Fit Model 
svr.fit(X_train, y_train)
# Assume user input
test_input2 = np.array([15, 1, 0, 3, 1, 1],dtype=object).reshape(1,6)
# Predicting the output
svr.predict(test_input2)

KNeighborsRegressor


array([2.11])

In [27]:
# Selecting the best model based on MSE
best_mse_model = min(model_scores, key=lambda x: x[3])
best_model_name = best_mse_model[0]
best_model_instance = models[best_model_name][0]
print(best_model_name)
# export 
pickle.dump(best_model_instance,open('pipe.pkl','wb'))
# Load Model 
kn = pickle.load(open('pipe.pkl','rb'))
# Fit Model 
kn.fit(X_train, y_train)
# Assume user input
test_input2 = np.array([15, 1, 0, 3, 1, 1],dtype=object).reshape(1,6)
# Predicting the output
kn.predict(test_input2)

XGBRegressor


array([2.5676944], dtype=float32)

- Using GridSeachCv The Best Performing Model based on `MAE and R2 ` is `SVR` and One Basis Of MSE `XGBRegressor is Best ` On `Tips` Dataset 
- Best Parameter For `SVR` is `{'Kernel': 'poly}`

---
# 2.Classification

In [28]:
# Load Data
df = sns.load_dataset('diamonds')
df = df.sample(1000)
# select features and variables
X = df.drop('cut', axis=1)
y = df['cut']

# label encode categorical variables
le = LabelEncoder()
X['color'] = le.fit_transform(X['color'])
X['clarity'] = le.fit_transform(X['clarity'])
# encode y 
le = LabelEncoder()
y = le.fit_transform(y)
# split the data into train and test data with 80% training dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 2.1 RandomizedCvSearch

In [29]:
%%time
# # Create a dictionaries of list of models to evaluate performance with hyperparameters
models = { 
          'LogicRegression' : (LogisticRegression(), {}),
          'SVC' : (SVC(), {'kernel': ['rbf', 'poly', 'sigmoid']}),
          'DesicionTreeClassifier' : (DecisionTreeClassifier(random_state=42), {'max_depth': [None, 5, 10],'random_state': [42]}),
          'RandomForestClassifier' : (RandomForestClassifier(random_state=42), {'n_estimators': [10, 100],'random_state': [42],'max_depth': [None, 5, 10]}),
          'KNeighborsClassifier' : (KNeighborsClassifier(), {'n_neighbors': np.arange(3, 100, 2),}),
          'GradientBoostingClassifier' : (GradientBoostingClassifier(random_state=42),{'n_estimators': [10, 100],'random_state': [42]}),
          'XGBClassifier' : (XGBClassifier(), {'n_estimators': [10, 100]}),  
          'AdaBoostClassifier': (AdaBoostClassifier(random_state=42), {'n_estimators': [10, 100],'random_state': [42]}),
          }

# train and predict each model with evaluation metrics as well making a for loop to iterate over the models
model_scores = []
for name, (model, params) in models.items():
    # create a pipline
    pipeline =RandomizedSearchCV(model, params, cv=5)
    
    # fit the pipeline
    pipeline.fit(X_train, y_train)
    
    # make prediction from each model
    y_pred = pipeline.predict(X_test)
    # Metric
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred,average='macro')
    precision = precision_score(y_test, y_pred,average='macro')
    best_parameter = pipeline.best_params_
    # Append these in the model_scores
    model_scores.append((name,accuracy , f1 , precision,best_parameter))                                    

# selecting the best model from all above models with evaluation metrics sorting method
sorted_models = sorted(model_scores, key=lambda x: x[1])

# Printing Each model with evaluation metrics
for model in sorted_models:
    print('Accuracy_score for', f"{model[0]} is {model[1]: .2f}")
print('\n')
for model in sorted_models:
    print('F1_Score for', f"{model[0]} is {model[2]: .2f}")
print('\n')
for model in sorted_models:
    print('Precision for', f"{model[0]} is {model[3]: .2f}")


Accuracy_score for KNeighborsClassifier is  0.36
Accuracy_score for SVC is  0.40
Accuracy_score for LogicRegression is  0.40
Accuracy_score for AdaBoostClassifier is  0.55
Accuracy_score for DesicionTreeClassifier is  0.70
Accuracy_score for RandomForestClassifier is  0.71
Accuracy_score for GradientBoostingClassifier is  0.72
Accuracy_score for XGBClassifier is  0.72


F1_Score for KNeighborsClassifier is  0.17
F1_Score for SVC is  0.13
F1_Score for LogicRegression is  0.16
F1_Score for AdaBoostClassifier is  0.45
F1_Score for DesicionTreeClassifier is  0.71
F1_Score for RandomForestClassifier is  0.73
F1_Score for GradientBoostingClassifier is  0.74
F1_Score for XGBClassifier is  0.71


Precision for KNeighborsClassifier is  0.18
Precision for SVC is  0.16
Precision for LogicRegression is  0.15
Precision for AdaBoostClassifier is  0.41
Precision for DesicionTreeClassifier is  0.75
Precision for RandomForestClassifier is  0.77
Precision for GradientBoostingClassifier is  0.79
Precisio

- Best Parameter and Best Model 

In [30]:
# Best Parameter for each model
for model in sorted_models:
    print('Best Parameters for', f"{model[0]} is {model[4]}")
print('\n')
# Selecting the best model based on MAE
best_Accuracy_model = max(model_scores, key=lambda x: x[1])
print('Best model based on Accuracy is', f"{best_Accuracy_model[0]} with Accuracy of {best_Accuracy_model[1]:.2f}")

# Selecting the best model based on R2
best_f1_model = max(model_scores, key=lambda x: x[2])
print('Best model based on F1_Score is', f"{best_f1_model[0]} with F1_Score of {best_f1_model[2]:.2f}")

# Selecting the best model based on MSE 
best_Precision_model = max(model_scores, key=lambda x: x[3])
print('Best model based on Precision is', f"{best_Precision_model[0]} with Precision of {best_Precision_model[3]:.2f}")

Best Parameters for KNeighborsClassifier is {'n_neighbors': 29}
Best Parameters for SVC is {'kernel': 'poly'}
Best Parameters for LogicRegression is {}
Best Parameters for AdaBoostClassifier is {'random_state': 42, 'n_estimators': 10}
Best Parameters for DesicionTreeClassifier is {'random_state': 42, 'max_depth': 5}
Best Parameters for RandomForestClassifier is {'random_state': 42, 'n_estimators': 100, 'max_depth': 10}
Best Parameters for GradientBoostingClassifier is {'random_state': 42, 'n_estimators': 10}
Best Parameters for XGBClassifier is {'n_estimators': 10}


Best model based on Accuracy is GradientBoostingClassifier with Accuracy of 0.72
Best model based on F1_Score is GradientBoostingClassifier with F1_Score of 0.74
Best model based on Precision is GradientBoostingClassifier with Precision of 0.79


- Using RandomizedCvSearch The Best Performing Model is `GradientBoostingClassifier`, `AdaBoostClassifier` and `XGBClassifier` On `Diamonds` Dataset 
- Best Parameter For `GradientBoostingClassifier` is `{'n_estimator': 10}`

In [31]:
# Selecting Best Model based on Accuracy
best_Accuracy_model = max(model_scores, key=lambda x: x[1])
best_model_name = best_Accuracy_model[0]
best_model_instance = models[best_model_name][0]
print(best_model_name)
# export
pickle.dump(best_model_instance,open('Classifier.pkl','wb'))
# Load Model
dtc = pickle.load(open('Classifier.pkl','rb'))
# Fit Model
dtc.fit(X_train, y_train)
# Predicting the output
y_pred = dtc.predict(X_test)
# Accuracy 
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy_score for', f"{best_model_name} is {accuracy: .2f}")
# Assume user input
test_input2 = np.array([0.51,0,5,61.5,56.0,1674,5.21,5.17,3.19],dtype=object).reshape(1,9)
# Predicting the output
dtc.predict(test_input2)

GradientBoostingClassifier
Accuracy_score for GradientBoostingClassifier is  0.73


array([2])

In [32]:
# Selecting Best Model based on F!_Score
best_f1_model = max(model_scores, key=lambda x: x[2])
best_model_name = best_f1_model[0]
best_model_instance = models[best_model_name][0]
print(best_model_name)
# export
pickle.dump(best_model_instance,open('Classifier_f1.pkl','wb'))
# Load Model
dtc = pickle.load(open('Classifier_f1.pkl','rb'))
# Fit Model
dtc.fit(X_train, y_train)
# Predicting the output
y_pred = dtc.predict(X_test)
# f1 
f1 = f1_score(y_test, y_pred, average='macro')
print('f1_score for', f"{best_model_name} is {f1: .2f}")
# Assume user input
test_input2 = np.array([0.51,0,5,61.5,56.0,1674,5.21,5.17,3.19],dtype=object).reshape(1,9)
# Predicting the output
dtc.predict(test_input2)

GradientBoostingClassifier
f1_score for GradientBoostingClassifier is  0.76


array([2])

## 2.2 Grid Search CV

In [33]:
%%time
# # Create a dictionaries of list of models to evaluate performance with hyperparameters
models = { 
          'LogicRegression' : (LogisticRegression(), {}),
          'SVC' : (SVC(), {'kernel': ['rbf', 'poly', 'sigmoid']}),
          'DesicionTreeClassifier' : (DecisionTreeClassifier(random_state=42), {'max_depth': [None, 5, 10],'random_state': [42]}),
          'RandomForestClassifier' : (RandomForestClassifier(random_state=42), {'n_estimators': [10, 100],'random_state': [42],'max_depth': [None, 5, 10]}),
          'KNeighborsClassifier' : (KNeighborsClassifier(), {'n_neighbors': np.arange(3, 100, 2),}),
          'GradientBoostingClassifier' : (GradientBoostingClassifier(random_state=42),{'n_estimators': [10, 100],'random_state': [42]}),
          'XGBClassifier' : (XGBClassifier(), {'n_estimators': [10, 100]}),  
          'AdaBoostClassifier': (AdaBoostClassifier(random_state=42), {'n_estimators': [10, 100],'random_state': [42]}),
          }

# train and predict each model with evaluation metrics as well making a for loop to iterate over the models
model_scores = []
for name, (model, params) in models.items():
    # create a pipline
    pipeline =GridSearchCV(model, params, cv=5)
    
    # fit the pipeline
    pipeline.fit(X_train, y_train)
    
    # make prediction from each model
    y_pred = pipeline.predict(X_test)
    # Metric
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred,average='macro')
    precision = precision_score(y_test, y_pred,average='macro')
    best_parameter = pipeline.best_params_
    # Append these in the model_scores
    model_scores.append((name,accuracy , f1 , precision,best_parameter))

# selecting the best model from all above models with evaluation metrics sorting method
sorted_models = sorted(model_scores, key=lambda x: x[1])

# Printing Each model with evaluation metrics
for model in sorted_models:
    print('Accuracy_score for', f"{model[0]} is {model[1]: .2f}")
print('\n')
for model in sorted_models:
    print('F1_Score for', f"{model[0]} is {model[2]: .2f}")
print('\n')
for model in sorted_models:
    print('Precision for', f"{model[0]} is {model[3]: .2f}")

Accuracy_score for KNeighborsClassifier is  0.39
Accuracy_score for SVC is  0.40
Accuracy_score for LogicRegression is  0.40
Accuracy_score for AdaBoostClassifier is  0.55
Accuracy_score for DesicionTreeClassifier is  0.70
Accuracy_score for RandomForestClassifier is  0.71
Accuracy_score for GradientBoostingClassifier is  0.72
Accuracy_score for XGBClassifier is  0.72


F1_Score for KNeighborsClassifier is  0.19
F1_Score for SVC is  0.13
F1_Score for LogicRegression is  0.16
F1_Score for AdaBoostClassifier is  0.45
F1_Score for DesicionTreeClassifier is  0.71
F1_Score for RandomForestClassifier is  0.73
F1_Score for GradientBoostingClassifier is  0.74
F1_Score for XGBClassifier is  0.71


Precision for KNeighborsClassifier is  0.27
Precision for SVC is  0.16
Precision for LogicRegression is  0.15
Precision for AdaBoostClassifier is  0.41
Precision for DesicionTreeClassifier is  0.75
Precision for RandomForestClassifier is  0.77
Precision for GradientBoostingClassifier is  0.79
Precisio

- Best Parameter and Best Model 

In [34]:
# Best Parameter for each model
for model in sorted_models:
    print('Best Parameters for', f"{model[0]} is {model[4]}")
print('\n')
# Selecting the best model based on MAE
best_Accuracy_model = max(model_scores, key=lambda x: x[1])
print('Best model based on Accuracy is', f"{best_Accuracy_model[0]} with Accuracy of {best_Accuracy_model[1]:.2f}")

# Selecting the best model based on R2
best_f1_model = max(model_scores, key=lambda x: x[2])
print('Best model based on F1_Score is', f"{best_f1_model[0]} with F1_Score of {best_f1_model[2]:.2f}")

# Selecting the best model based on MSE 
best_Precision_model = max(model_scores, key=lambda x: x[3])
print('Best model based on Precision is', f"{best_Precision_model[0]} with Precision of {best_Precision_model[3]:.2f}")

Best Parameters for KNeighborsClassifier is {'n_neighbors': 25}
Best Parameters for SVC is {'kernel': 'poly'}
Best Parameters for LogicRegression is {}
Best Parameters for AdaBoostClassifier is {'n_estimators': 10, 'random_state': 42}
Best Parameters for DesicionTreeClassifier is {'max_depth': 5, 'random_state': 42}
Best Parameters for RandomForestClassifier is {'max_depth': 10, 'n_estimators': 100, 'random_state': 42}
Best Parameters for GradientBoostingClassifier is {'n_estimators': 10, 'random_state': 42}
Best Parameters for XGBClassifier is {'n_estimators': 10}


Best model based on Accuracy is GradientBoostingClassifier with Accuracy of 0.72
Best model based on F1_Score is GradientBoostingClassifier with F1_Score of 0.74
Best model based on Precision is GradientBoostingClassifier with Precision of 0.79


- Using GridSreachCv The Best Performing Model is `GradientBoostingClassifier` ,`AdaBoostClassifier` and `XGBClassifier` On `Diamonds` Dataset 
- Best Parameters for XGBClassifier is {'n_estimators': 10}
- Best Parameters for GradientBoostingClassifier is {'n_estimators': 10}
- Best Parameters for AdaBoostClassifier is {'n_estimators': 10}

----

In [35]:
df = sns.load_dataset('tips')

# select features and variables
X = df.drop('tip', axis=1)
y = df['tip']

# label encode categorical variables
le = LabelEncoder()
X['sex'] = le.fit_transform(X['sex'])
X['smoker'] = le.fit_transform(X['smoker'])
X['day'] = le.fit_transform(X['day'])
X['time'] = le.fit_transform(X['time'])

# split the data into train and test data with 80% training dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a dictionaries of list of models to evaluate performance
models = { 
          'LinearRegression' : LinearRegression(),
          'SVR' : SVR(),
          'DecisionTreeRegressor' : DecisionTreeRegressor(random_state=42),
          'RandomForestRegressor' : RandomForestRegressor(random_state=42),
          'KNeighborsRegressor' : KNeighborsRegressor(),
          'GradientBoostingRegressor' : GradientBoostingRegressor(random_state=42),
          'XGBRegressor' : XGBRegressor()  ,
          'AdaBoostRegressor' : AdaBoostRegressor(random_state=42)       
          }

# train and predict each model with evaluation metrics as well making a for loop to iterate over the models

model_scores = []
for name, model in models.items():
    # fit each model from models on training data
    model.fit(X_train, y_train)
    
    # make prediction from each model
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    # metric = r2_score(y_test, y_pred)
    model_scores.append((name, mae , r2 , mse))
    
      
    # # print the performing metric
    # print(name, 'MSE: ', mean_squared_error(y_test, y_pred))
    # print(name, 'R2: ', r2_score(y_test, y_pred))
    # print(name, 'MAE: ', mean_absolute_error(y_test, y_pred))
    # print('\n')
# selecting the best model from all above models with evaluation metrics sorting method
sorted_models = sorted(model_scores, key=lambda x: x[1], reverse=False)
for model in sorted_models:
    print('Mean Absolute error for', f"{model[0]} is {model[1]: .2f}")
print('===============================================================')
for model in sorted_models:
    print('R2 for', f"{model[0]} is {model[2]: .2f}")
print('===============================================================')

for model in sorted_models:
    print('MSE for', f"{model[0]} is {model[3]: .2f}") 
print('===============================================================')

# Selecting the best model based on MAE
best_mae_model = min(model_scores, key=lambda x: x[1])
print('Best model based on MAE is', f"{best_mae_model[0]} with MAE of {best_mae_model[1]:.2f}")

# Selecting the best model based on R2
best_r2_model = max(model_scores, key=lambda x: x[2])
print('Best model based on R2 is', f"{best_r2_model[0]} with R2 of {best_r2_model[2]:.2f}")

# Selecting the best model based on MSE 
best_mse_model = min(model_scores, key=lambda x: x[3])
print('Best model based on MSE is', f"{best_mse_model[0]} with MSE of {best_mse_model[3]:.2f}")



Mean Absolute error for SVR is  0.57
Mean Absolute error for LinearRegression is  0.67
Mean Absolute error for XGBRegressor is  0.67
Mean Absolute error for GradientBoostingRegressor is  0.73
Mean Absolute error for KNeighborsRegressor is  0.73
Mean Absolute error for RandomForestRegressor is  0.78
Mean Absolute error for AdaBoostRegressor is  0.81
Mean Absolute error for DecisionTreeRegressor is  0.92
R2 for SVR is  0.57
R2 for LinearRegression is  0.44
R2 for XGBRegressor is  0.41
R2 for GradientBoostingRegressor is  0.36
R2 for KNeighborsRegressor is  0.33
R2 for RandomForestRegressor is  0.23
R2 for AdaBoostRegressor is  0.21
R2 for DecisionTreeRegressor is -0.13
MSE for SVR is  0.54
MSE for LinearRegression is  0.69
MSE for XGBRegressor is  0.74
MSE for GradientBoostingRegressor is  0.80
MSE for KNeighborsRegressor is  0.84
MSE for RandomForestRegressor is  0.96
MSE for AdaBoostRegressor is  0.99
MSE for DecisionTreeRegressor is  1.41
Best model based on MAE is SVR with MAE of 0.5